![facs1/4](https://img.shields.io/badge/facs1/4-lightgrey)
[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/facs.ipynb)
[![lamindata](https://img.shields.io/badge/Source%20%26%20report%20on%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/transform/OWuTtS4SApon5zKv/1p2uQHNQJvGmld3ZarEQ)

# Flow cytometry

You'll learn how to manage a growing number of flow cytometry datasets as a single queryable collection.

Specifically, you will

1. read a single `.fcs` file as an `AnnData` and seed a versioned collection with it (![facs1/4](https://img.shields.io/badge/facs1/4-lightgrey), current page)
2. append a new dataset (a new `.fcs` file) to create a new version of the collection ([![facs2/4](https://img.shields.io/badge/facs2/4-lightgrey)](facs2))
3. query individual files and cell markers ([![facs3/4](https://img.shields.io/badge/facs3/4-lightgrey)](facs3))
4. analyze the collection and store results as plots ([![facs4/4](https://img.shields.io/badge/facs4/4-lightgrey)](facs4))

```{toctree}
:maxdepth: 1
:hidden:

facs2
facs3
facs4
```

In [ ]:
# !pip install 'lamindb[jupyter,bionty]'
!lamin init --storage ./test-facs --schema bionty

In [ ]:
import lamindb as ln
import bionty as bt
import readfcs

bt.settings.organism = "human"  # globally set organism to human

ln.track("OWuTtS4SApon0000")

## Ingest a first artifact

### Access ![](https://img.shields.io/badge/Access-10b981)

We start with a flow cytometry file from [Alpert *et al.*, Nat. Med. (2019)](https://pubmed.ncbi.nlm.nih.gov/30842675/).

Calling the following function downloads the artifact and pre-populates a few relevant registries:

In [ ]:
ln.core.datasets.file_fcs_alpert19(populate_registries=True)

We use [readfcs](https://lamin.ai/docs/readfcs) to read the raw fcs file into memory and create an `AnnData` object:

In [ ]:
adata = readfcs.read("Alpert19.fcs")
adata

It has the following features:

In [ ]:
adata.var.head(10)

### Transform: normalize ![](https://img.shields.io/badge/Transform-10b981)

In this use case, we'd like to ingest & store curated data, and hence, we split signal and normalize using the [pytometry](https://github.com/buettnerlab/pytometry) package.

In [ ]:
import pytometry as pm

First, we'll split the signal from heigh and area metadata:

In [ ]:
pm.pp.split_signal(adata, var_key="channel", data_type="cytof")

In [ ]:
adata

Normalize the collection:

In [ ]:
pm.tl.normalize_arcsinh(adata, cofactor=150)

```{note}

If the collection was a flow collection, you'll also have to compensate the data, if possible. The metadata should contain a compensation matrix, which could then be run by the pytometry compensation function. In the case here, its a cyTOF collection, which doesn't (really) require compensation. 

```

### Validate: cell markers ![](https://img.shields.io/badge/Validate-10b981)

First, we validate features in `.var` using {class}`~docs:bionty.CellMarker`:

In [ ]:
validated = bt.CellMarker.validate(adata.var.index)

We see that many features aren't validated because they're not standardized.

Hence, let's standardize feature names & validate again:

In [ ]:
adata.var.index = bt.CellMarker.standardize(adata.var.index)
validated = bt.CellMarker.validate(adata.var.index)

The remaining non-validated features don't appear to be cell markers but rather metadata features.

Let's move them into `adata.obs`:

In [ ]:
adata.obs = adata[:, ~validated].to_df()
adata = adata[:, validated].copy()

Now we have a clean panel of 35 validated cell markers:

In [ ]:
validated = bt.CellMarker.validate(adata.var.index)
assert all(validated)  # all markers are validated

### Register: metadata ![](https://img.shields.io/badge/Register-10b981)

Next, let's register the metadata features we moved to `.obs`.

For this, we create one feature record for each column in the `.obs` dataframe:

In [ ]:
features = ln.Feature.from_df(adata.obs)
ln.save(features)

We use the [Experimental Factor Ontology](https://www.ebi.ac.uk/efo/) through Bionty to create a "FACS" label:

In [ ]:
bt.ExperimentalFactor.public().search("FACS").head(2)  # search the public ontology

We found one for "FACS", let's save it to our in-house registry:

In [ ]:
# import the FACS record from the public ontology and save it to the registry
facs = bt.ExperimentalFactor.from_source(ontology_id="EFO:0009108")
facs.save()

 We don't find one for "CyToF", however, so, let's create it without importing from a public ontology but label it as a child of "is_cytometry_assay":

In [ ]:
cytof = bt.ExperimentalFactor(name="CyTOF")
cytof.save()
is_cytometry_assay = bt.ExperimentalFactor(name="is_cytometry_assay")
is_cytometry_assay.save()
cytof.parents.add(is_cytometry_assay)
facs.parents.add(is_cytometry_assay)

is_cytometry_assay.view_parents(with_children=True)

Let us look at the content of the registry:

In [ ]:
bt.ExperimentalFactor.df()

### Register: save & annotate with metadata ![](https://img.shields.io/badge/Register-10b981)

In [ ]:
curate = ln.Curator.from_anndata(adata, var_index=bt.CellMarker.name, categoricals={})
curate.validate()

In [ ]:
artifact = curate.save_artifact(description="Alpert19")

Add more labels:

In [ ]:
experimental_factors = bt.ExperimentalFactor.lookup()
organisms = bt.Organism.lookup()

artifact.labels.add(experimental_factors.cytof)
artifact.labels.add(organisms.human)

## Inspect the saved artifact

Inspect features on a high level:

In [ ]:
artifact.features

Inspect low-level features in `.var`:

In [ ]:
artifact.features["var"].df().head()

Use auto-complete for marker names in the `var` featureset:

In [ ]:
markers = artifact.features["var"].lookup()
markers.cd14

In a plot, we can now easily also show gene symbol and Uniprot ID:

In [ ]:
import scanpy as sc

sc.pp.pca(adata)
sc.pl.pca(
    adata,
    color=markers.cd14.name,
    title=(
        f"{markers.cd14.name} / {markers.cd14.gene_symbol} /"
        f" {markers.cd14.uniprotkb_id}"
    ),
)

## Create a collection from the artifact

In [ ]:
ln.Collection(artifact, key="My versioned cytometry collection", version="1").save()